In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re


driver = webdriver.Chrome('/home/eunbi/Desktop/Study/크롤링/chromedriver')
driver.implicitly_wait(10)
driver.get('https://www.kurly.com/shop/main/index.php')
driver.implicitly_wait(10)


for i in range(1, 16):
    # 전체 카테고리 클릭
    driver.find_element_by_css_selector(
    '#gnbMenu > div > div.gnb_main > ul > li.menu1 > a').click()
    driver.implicitly_wait(10)
    # 카테고리 클릭
    driver.find_element_by_css_selector(
        f'#gnbMenu > div > div.gnb_sub > div > ul > li:nth-child({i}) > a > span.tit > span').click()
    driver.implicitly_wait(10)
    subcategory_url = driver.current_url
    
    # 서브카테고리 클릭
    for s in range(2, 10):
        try:
            driver.find_element_by_css_selector(
            f'#lnbMenu > div.inner_lnb > ul > li:nth-child({s}) > a').click()
        except:
            break
        time.sleep(1)
        subcategory_name =  driver.find_element_by_css_selector(
            f'#lnbMenu > div.inner_lnb > ul > li:nth-child({s}) > a')
        print(subcategory_name.text)
        
        # 제품 상세 화면 클릭
        for g in range(1, 17):
            driver.find_element_by_css_selector(
            f'#goodsList > div.list_goods > div > ul > li:nth-child({g}) > div > a').click()
            driver.implicitly_wait(20)
            # soup 설정
            link = driver.current_url
            driver.get(link)
            time.sleep(1)
            html = driver.page_source
            time.sleep(1)
            soup = BeautifulSoup(html, 'html.parser')
            time.sleep(1)

            
            # 썸네일
            img1 = soup.select_one('#sectionView > div > div.thumb > img')
            print(f"img1: {img1['src']}")

            
            # 상품명
            name = soup.select_one('#sectionView > div > p.goods_name > strong').string
            print(f'name: {name}')

            # 짧은설명
            summary = soup.select_one('#sectionView > div > p.goods_name > span.short_desc').string
            print(f'summary: {summary}')

            # 가격
            price = soup.select_one(
            '#sectionView > div > p.goods_price > span.position > span.dc > span.dc_price').get_text()
            print(f'price: {price}')

            ## 할인
            try:
                discount_rate = soup.select_one(
                '#sectionView > div > p.goods_price > span.position > span.dc > span.dc_percent').string
                print(f'discount_rate: {discount_rate}')
            except:
                pass
            
            goods_info = soup.select_one('#sectionView > div > div.goods_info')
            unit, amount, package, made_in = '', '', '', ''
            for dl in goods_info:
                try:
                    dt = dl.select_one('dt.tit')
                    dd = dl.select_one('dd.desc')
                    if dt.text == '판매단위':
                        unit = dd.text
                        continue
                    elif dt.text == '중량/용량':
                        amount = dd.text
                        continue
                    elif dt.text == '원산지':
                        made_in = dd.text.strip()
                        continue
                    elif dt.text == '포장타입':
                        for d in dd:
                            package = d.string.strip()
                            break
                        continue
                except:
                    pass
            print(f'unit: {unit}, amount: {amount}, made_in: {made_in}, package: {package}')
            
            # 큰 이미지

            img2 = soup.select_one(
                '#goods-description > div > div.goods_intro > div.pic > img')
            img2 = img2['src']
            img2 = 'https:'+img2
            print(f'img2: {img2}')

            # 상품설명
            description = soup.select_one(
                '#goods-description > div > div.goods_intro > div.context.last > p').string
            print(f'description: {description}')
            
            # 상품이미지
            img3 = soup.select_one(
                '#goods_pi > p > img')
            img3 = img3['src']
            img3 = 'https:'+img3
            print(f'img3: {img3}')
            time.sleep(1)
            
            # options
            try:
                option_select = driver.find_element_by_class_name("txt_select").click()
                option_url = driver.current_url
                driver.get(option_url)
                html = driver.page_source
                time.sleep(2)
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(2)
                options = soup.select_one('#cartPut > div.cart_option.cart_type2 > div > div.in_option > div.list_goods > div > div > ul')
                for li in options:
                    for span in li.a:
                        option_name = span.text.strip()
                        break
                    option_price = li.find("span", class_="dc_price").text.strip()
                    print(f'option_name: {option_name}')
                    print(f'option_price: {option_price}')
            except:
                pass    
            print('----------------------------------------------------------------------')
            # 서브카테고리 메뉴로 가기
            driver.get(subcategory_url) 
            time.sleep(1)


해산물·조개류
img1: https://res.kurly.com/mobile/service/goodsview/1910/bg_375x482.png
name: [아라움] 오징어 몸통 120g
summary: 부드럽게 씹히는
price: 12,900원
unit: 1팩, amount: 120g, made_in: 국산, package: 상온/종이포장
img2: https://img-cf.kurly.com/shop/data/goodsview/20200331/gv00000088143_1.jpg
description: 쫄깃하게 말려 감칠맛이 한층 깊어진 건어포는 편하게 즐길 수 있으면서 자꾸만 손이 가는 별미 간식이지요. 다양한 생김새만큼이나 맛과 즐기는 법까지 각양각색인 아라움의 건어포를 소개할게요. 지금 만나볼 제품은 껍질을 제거한 국내산 오징어 몸통입니다. 완전히 마르지 않은 촉촉한 상태로, 질기지 않아 씹기에 부담이 없어요. 튀김옷을 입혀 기름에 튀기거나 칼집을 내 버터를 발라 구워도 별미이니, 다양한 요리에 활용해 보세요. 
img3: https://img-cf.kurly.com/shop/data/goodsview/20200331/gv00000088251_1.jpg
----------------------------------------------------------------------
img1: https://res.kurly.com/mobile/service/goodsview/1910/bg_375x482.png
name: [윤슬] 오만둥이 180g(냉장)
summary: 쫀득쫀득 경쾌한 식감
price: 3,900원
unit: 1팩, amount: 180g, made_in: 국산, package: 냉장/종이포장
img2: https://img-cf.kurly.com/shop/data/goodsview/20200325/gv40000087157_1.jpg
description: 경상도 방언으로 '오만 데에(온갖 곳에) 다 붙어 있다'고 해서 오만둥이라는 이름이 

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=36731): Max retries exceeded with url: /session/e363798be81bfc5488049a2269f73997/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f6d4bcfa898>: Failed to establish a new connection: [Errno 111] Connection refused',))